# Programming Homework 2

### Question 1
How many alignments does the naive exact matching algorithm try when matching the string **GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG** (derived from human Alu sequences) to the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [1]:
from bm_with_counts import boyer_moore_with_counts
from bm_preproc import BoyerMoore
from naive_with_counts import naive_with_counts
import wget

In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
chr1=readGenome('chr1.GRCh38.excerpt.fasta')
len(chr1)

800000

In [4]:
p='GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t=chr1
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print (occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


### Question 2
How many character comparisons does the naive exact matching algorithm try when matching the string **GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG** (derived from human Alu sequences) to the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [5]:
print (occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


### Question 3
How many alignments does Boyer-Moore try when matching the string **GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG** (derived from human Alu sequences) to the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [6]:
p='GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t=chr1
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm ,t)
print (occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


### Question 4
Index-assisted approximate matching. In practicals, we built a Python class called **Index**
implementing an ordered-list version of the k-mer index. The **Index** class is copied below.

In [7]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        import bisect
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.
Implement the pigeonhole principle using **Index** to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an **Index** object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string **GGCGCGGTGGCTCACGCCTGTAAT**, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1? (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.
Hint 2: You can check your work by comparing the output of your new function to that of the **naive_2mm** function implemented in the previous module.

In [8]:
def approximate_match(p, t, n, index):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    indexhit=0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        # Extend matching segments to see if whole p matches
        for m in matches:
            indexhit += 1
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), indexhit

In [9]:
p='GGCGCGGTGGCTCACGCCTGTAAT'
t=chr1
index=Index(t,8)
matches,indexhit=approximate_match(p, t, 2,index)
print(len(matches),indexhit)


19 90


### Question 5
Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of **GGCGCGGTGGCTCACGCCTGTAAT** with up to 2 substitutions in the excerpt of human chromosome 1?

(Don't consider reverse complements.)

Hint: You should be able to use the **boyer_moore function** (or the slower **naive** function) to double-check your answer.

In [10]:
print(indexhit)

90


Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video. We'll consider subsequences involving every N characters. For example, if we split **ATATAT** into two substring partitions, we would get partitions **ATA** (the first half) and **TAT** (second half). But if we split **ATATAT** into two subsequences by taking every other character, we would get **AAA** (first, third and fifth characters) and **TTT** (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition. Splitting a length-6 pattern into two substrings could be represented as **111222**, and splitting into two subsequences of every other character could be represented as **121212**

The following class **ubseqIndex** is a more general implementation of **Index** that additionally handles subsequences. It only considers subsequences that take every Nth character:

In [11]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


For example, if we do:


In [12]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


And if we query this index:

In [13]:
p = 'TTATAT'
print(ind.query(p[0:]))

[]


because the subsequence **TAA** is not in the index. But if we query with the second subsequence:

In [14]:
print(ind.query(p[1:]))

[1]


because the second subsequence **TTT** is in the index.

### Question 7
Write a function that, given a length-24 pattern P and given a **SubseqIndex** object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for **GGCGCGGTGGCTCACGCCTGTAAT** with up to 2 substitutions in the excerpt of human chromosome 1? (Again, don't consider reverse complements.)

Hint: See this notebook for a few examples you can use to test your function.

In [15]:
p='GGCGCGGTGGCTCACGCCTGTAAT'
t=chr1
index=SubseqIndex(t, 3, 2)
matches,indexhit=approximate_match(p, t, 2,index)
print(len(matches),indexhit)

34 26610


In [16]:
def query_subseq(p, t, index):
    k = index.k
    offsets = []
    hit=0
    for i in index.query(p):
        hit+=1
        if p[0:] == t[i+0:i+len(p)]:  # verify that rest of P matches
            offsets.append(i)
    for i in index.query(p[1:]):
        hit+=1
        if p[1:] == t[i+1:i+len(p)-1]:  # verify that rest of P matches
            offsets.append(i)
    for i in index.query(p[2:]):
        hit+=1
        if p[2:] == t[i+2:i+len(p)-2]:  # verify that rest of P matches
            offsets.append(i)            
    return offsets,hit 

In [17]:
t = chr1
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = query_subseq(p, t, subseq_ind)

In [18]:
print(occurrences)
print(num_index_hits)

[56922, 262042, 364263, 657496, 717706]
79
